In [2]:
import scrapy

In [3]:
class spider12(scrapy.Spider): #Spider es el scrapper generico de scrapy
    name = 'spider12'
    allowed_domains = ['pagina12.com.ar']
    
    custom_settings = {'FEED_FORMAT':'json',
                      'FEED_URI':'resultados.json',
                      'DEPTH_LIMIT':2}
    
    start_url = ['https://www.pagina12.com.ar/secciones/el-pais',
                 'https://www.pagina12.com.ar/secciones/economia',
                 'https://www.pagina12.com.ar/secciones/sociedad',
                 'https://www.pagina12.com.ar/suplementos/cultura-y-espectaculos',
                 'https://www.pagina12.com.ar/secciones/el-mundo',
                 'https://www.pagina12.com.ar/secciones/deportes',
                 'https://www.pagina12.com.ar/secciones/contratapa',
                 'https://www.pagina12.com.ar/secciones/audiovisuales']
    
    def parse(self, response):
        # Articulo Promocionado
        nota_promocionada = response.xpath('//div[@class="featured-article__container"]/a/@href').get()
        
        if nota_promocionada is not None:
            yield response.follow(nota_promocionada, callback = self.parse_nota) 
        
        #Listado de notas
        notas = response.xpath('//ul[@class="article-list"]//li//h2//a/@href').get_all()
        for nota in notas:
            yield response.follow(nota, callback = self.parse_nota)
        
        #Link a la siguiente pagina
        next_page = response.xpath('//a[@class="pagination-btn-next"]/@href').get()
        if next_page is not None:
            yield response.follow(next_page, callback = self.parse)
            
    def parse_nota(self, response):
        #Extraccion del titulo de la nota
        titulo = response.xpath('//div[@class="article-titles"]/text()').get()
        
        #Extraccion de la fecha de la nota
        fecha = response.xpath('//span[@pubdate="pubdate"]/@datetime').get()
        
        #Extraccion de la volanta de la nota
        volanta = response.xpath('//h2[@class="article-prefix"]/text()').get()
        
        #Extraccion del cuerpo de la nota
        cuerpo = ''.join(response.xpath('//div[@class="article-text"]/b/text()')).getall()
        
        #Extraccion del autor de la nota 
        autor = response.xpath('//div[@class="article-author"]/a/text()').get()
        
        yield {'url':response.url,
              'titulo':titulo,
              'fecha':fecha,
              'volanta':volanta,
              'cuerpo':cuerpo,
              'autor':autor}

In [4]:
from scrapy.crawler import CrawlerProcess

In [5]:
process = CrawlerProcess()
process.crawl(spider12)
process.start()

2020-04-11 18:35:21 [scrapy.utils.log] INFO: Scrapy 2.0.1 started (bot: scrapybot)
2020-04-11 18:35:21 [scrapy.utils.log] INFO: Versions: lxml 4.5.0.0, libxml2 2.9.9, cssselect 1.1.0, parsel 1.5.2, w3lib 1.21.0, Twisted 20.3.0, Python 3.7.6 (default, Jan  8 2020, 20:23:39) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 19.1.0 (OpenSSL 1.1.1d  10 Sep 2019), cryptography 2.8, Platform Windows-10-10.0.18362-SP0
2020-04-11 18:35:21 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2020-04-11 18:35:21 [scrapy.crawler] INFO: Overridden settings:
{'DEPTH_LIMIT': 2, 'FEED_FORMAT': 'json', 'FEED_URI': 'resultados.json'}
2020-04-11 18:35:21 [scrapy.extensions.telnet] INFO: Telnet Password: c720132dd277a295
2020-04-11 18:35:21 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2020-04-11 18:35:21 [scrapy.m